<a href="https://colab.research.google.com/github/goto0823/dog_cat/blob/main/dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

data_dir = "/content/detasets/"

IMG_HEIGHT = 150
IMG_WIDTH  = 150

BATCH_SIZE = 5

train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=4 # Set validation batch size to the size of the validation dataset
)

data_arg = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])

model = tf.keras.Sequential([
  data_arg,
  # 入力層（入力の形と、ピクセル値の正規化）
  tf.keras.layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

  # 畳み込みブロック1
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),

  # 畳み込みブロック2
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),

  # 畳み込みブロック3
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),

  # 最後の分類のための層
  tf.keras.layers.Flatten(), # ここで初めて平坦化する
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid') # 最終的な出力
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10
)

# 1. 画像を読み込み、指定したサイズにリサイズする
img = tf.keras.utils.load_img(
    "/content/data/cat2.jpeg", target_size=(IMG_HEIGHT, IMG_WIDTH)
)
# 2. 画像を数値の配列(NumPy array)に変換する
img_array = tf.keras.utils.img_to_array(img)
# 3. モデルは一度に複数の画像を処理する「バッチ」形式でデータを期待するため、
#    次元を一つ追加して (1, 150, 150, 3) という形にする
img_array = tf.expand_dims(img_array, 0) # Create a batch
# 4. モデルに画像データを渡して予測を実行！
predictions = model.predict(img_array)
score = predictions[0][0] # 予測結果は [[...]] のような形で返ってくるので、中の数値を取り出す
# 5. 結果を人間が分かりやすいように表示する
class_names = train_dataset.class_names # 学習データからクラス名['dog', 'not_dog']を取得
print(
    "この画像は {:.2f} %の確率で {} であり、{:.2f} %の確率で {} です。"
    .format(100 * (1 - score), class_names[0], 100 * score, class_names[1])
)
# 閾値（例えば0.5）を使って、最終的な判定を下す
if score < 0.5:
    print(f"最終的な判定: この画像は {class_names[0]} です！")
else:
    print(f"最終的な判定: この画像は {class_names[1]} です！")

Found 223 files belonging to 2 classes.
Using 179 files for training.
Found 223 files belonging to 2 classes.
Using 44 files for validation.
Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 166ms/step - accuracy: 0.4437 - loss: 1.0553 - val_accuracy: 0.5682 - val_loss: 0.6909
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 167ms/step - accuracy: 0.4546 - loss: 0.6947 - val_accuracy: 0.5909 - val_loss: 0.6913
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 159ms/step - accuracy: 0.4160 - loss: 0.6941 - val_accuracy: 0.5455 - val_loss: 0.6920
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 170ms/step - accuracy: 0.5773 - loss: 0.6928 - val_accuracy: 0.5682 - val_loss: 0.6850
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 194ms/step - accuracy: 0.4918 - loss: 0.6953 - val_accuracy: 0.5455 - val_loss: 0.6892
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 263ms/step - accuracy: 0.4854 - loss: 0.6916 - val_accuracy: 0.5682 - val_loss: 0.6742
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 158ms/step - accuracy: 0.5633 - loss: 0.6985 

In [ ]:
import os

data_dir = "/content/datasets/"

# Create the main directory
os.makedirs(data_dir, exist_ok=True)

# Create dummy class subdirectories
class_names = ["cat", "dog"]
for class_name in class_names:
    os.makedirs(os.path.join(data_dir, class_name), exist_ok=True)

# Create dummy image files (replace with your actual images)
# For demonstration, we'll just create empty files
for i in range(5):
    with open(os.path.join(data_dir, "cat", f"cat_{i}.jpg"), "w") as f:
        pass
    with open(os.path.join(data_dir, "dog", f"dog_{i}.jpg"), "w") as f:
        pass

print(f"Dummy dataset structure created in {data_dir}")

Dummy dataset structure created in /content/datasets/


In [ ]:
import tensorflow as tf

data_dir = "/content/datasets/"

IMG_HEIGHT = 150
IMG_WIDTH  = 150

BATCH_SIZE = 5

train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=4 # Set validation batch size to the size of the validation dataset
)

data_arg = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])

model = tf.keras.Sequential([
  data_arg,
  # 入力層（入力の形と、ピクセル値の正規化）
  tf.keras.layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

  # 畳み込みブロック1
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),

  # 畳み込みブロック2
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),

  # 畳み込みブロック3
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),

  # 最後の分類のための層
  tf.keras.layers.Flatten(), # ここで初めて平坦化する
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid') # 最終的な出力
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10
)

# 1. 画像を読み込み、指定したサイズにリサイズする
img = tf.keras.utils.load_img(
    "/content/datasets/cat/cat_0.jpg", target_size=(IMG_HEIGHT, IMG_WIDTH)
)
# 2. 画像を数値の配列(NumPy array)に変換する
img_array = tf.keras.utils.img_to_array(img)
# 3. モデルは一度に複数の画像を処理する「バッチ」形式でデータを期待するため、
#    次元を一つ追加して (1, 150, 150, 3) という形にする
img_array = tf.expand_dims(img_array, 0) # Create a batch
# 4. モデルに画像データを渡して予測を実行！
predictions = model.predict(img_array)
score = predictions[0][0] # 予測結果は [[...]] のような形で返ってくるので、中の数値を取り出す
# 5. 結果を人間が分かりやすいように表示する
class_names = train_dataset.class_names # 学習データからクラス名['dog', 'not_dog']を取得
print(
    "この画像は {:.2f} %の確率で {} であり、{:.2f} %の確率で {} です。"
    .format(100 * (1 - score), class_names[0], 100 * score, class_names[1])
)
# 閾値（例えば0.5）を使って、最終的な判定を下す
if score < 0.5:
    print(f"最終的な判定: この画像は {class_names[0]} です！")
else:
    print(f"最終的な判定: この画像は {class_names[1]} です！")

Found 10 files belonging to 2 classes.
Using 8 files for training.
Found 10 files belonging to 2 classes.
Using 2 files for validation.
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Input is empty.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_multi_step_on_iterator_93511]